In [1]:
import sys
sys.path.append('../')
import psycopg2
from configparser import ConfigParser
import requests
from datetime import datetime
from config.settings import settings


from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from selenium.common.exceptions import TimeoutException
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import psycopg2
from configparser import ConfigParser
import re
import logging
import time
import json

### Connect to the database

In [16]:
def connect(config):
    """ Connect to the PostgreSQL database server """
    try:
        # connecting to the PostgreSQL server
        with psycopg2.connect(**config) as conn:
            print('Connected to the PostgreSQL server.')
            return conn
    except (psycopg2.DatabaseError, Exception) as error:
        print(error)

In [17]:
def load_config(filename='../database/database.ini', section='postgresql'):
    parser = ConfigParser()
    parser.read(filename)

    # get section, default to postgresql
    config = {}
    if parser.has_section(section):
        params = parser.items(section)
        for param in params:
            config[param[0]] = param[1]
    else:
        raise Exception('Section {0} not found in the {1} file'.format(section, filename))

    return config


In [18]:
config = load_config()
connect(config)

Connected to the PostgreSQL server.


<connection object at 0x00000168FE8AFCD0; dsn: 'user=postgres password=xxx dbname=debris_flow_dt host=localhost', closed: 0>

In [22]:
def create_weather_table(config):
    """
    Create weather_data_hourly table if it doesn't exist
    """
    create_table_query = """
    CREATE TABLE IF NOT EXISTS weather_data_hourly (
        id SERIAL PRIMARY KEY,
        station_id VARCHAR(50) NOT NULL,
        timestamp TIMESTAMP NOT NULL,
        temperature_c FLOAT,
        humidity_percent FLOAT,
        pressure_hpa FLOAT,
        wind_speed_kmh FLOAT,
        gust_speed_kmh FLOAT,
        precipitation_mm FLOAT,
        created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
        data_source VARCHAR(50) DEFAULT 'AWEKAS_TABLE',
        UNIQUE(station_id, timestamp)
    );
    
    CREATE INDEX IF NOT EXISTS idx_station_timestamp 
    ON weather_data_hourly(station_id, timestamp);
    
    CREATE INDEX IF NOT EXISTS idx_timestamp 
    ON weather_data_hourly(timestamp);
    """
    
    try:
        with psycopg2.connect(**config) as conn:
            with conn.cursor() as cur:
                cur.execute(create_table_query)
                conn.commit()
                # logger.info("Table 'weather_data_hourly' created/verified")
                print("Table 'weather_data_hourly' created/verified")
                return True
    except (psycopg2.DatabaseError, Exception) as error:
        # logger.error(f"Error creating table: {error}")
        print(f"Error creating table: {error}")
        return False

### Wrap data from web
AWEKAS WEATHER DATA: Grossglockner Station

In [5]:
station_id = "34362"
base_url = "https://stationsweb.awekas.at"
urls = {
            'index-tab': f"{base_url}/en/{station_id}/index-tab",
            'table': f"{base_url}/en/{station_id}/table",
            'data': f"{base_url}/en/{station_id}/data",
            'statistic': f"{base_url}/en/{station_id}/statistic"
        }

In [6]:
# Setup Chrome options
chrome_options = Options()
chrome_options.add_argument('--headless')  # Run without GUI
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
chrome_options.add_argument('--disable-gpu')
chrome_options.add_argument('--window-size=1920,1080')
chrome_options.add_argument('user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36')

In [7]:
def accept_cookies_ionic(driver, timeout=10):
    try:
        # Wait until ion-modal is present
        WebDriverWait(driver, timeout).until(
            lambda d: d.execute_script(
                "return document.querySelector('ion-modal#cookie-banner') !== null"
            )
        )

        # Click "Accept all" inside Shadow DOM
        driver.execute_script("""
            const modal = document.querySelector('ion-modal#cookie-banner');
            if (!modal) return;

            const root = modal.shadowRoot;
            if (!root) return;

            const buttons = modal.querySelectorAll('ion-button');
            for (const btn of buttons) {
                if (btn.innerText.trim().toLowerCase().includes('accept')) {
                    btn.click();
                    return;
                }
            }
        """)
        print("Cookie banner accepted")

        # Small wait to allow modal to close
        time.sleep(0.5)

    except TimeoutException:
        print("No cookie banner found")


In [8]:
driver = webdriver.Chrome(options=chrome_options)
wait = WebDriverWait(driver, 15)
driver.get(urls['table'])
accept_cookies_ionic(driver)

Cookie banner accepted


In [ ]:
date_element = wait.until(
    EC.visibility_of_element_located(
        (By.XPATH, "//div[contains(@class,'date') and contains(@class,'visible')]//ion-text")
    )
)
date_text = date_element.text
date_obj = datetime.strptime(date_text, "%B %d, %Y").date()
print(date_obj)   #2026-01-20


2026-01-20


In [10]:
rows_count = len(
    driver.find_elements(
        By.XPATH,
        "//div[contains(@class,'card') and contains(@class,'visible')]//tbody/tr"
    )
)
day_data = []

for i in range(rows_count):
    rows = driver.find_elements(
        By.XPATH,
        "//div[contains(@class,'card') and contains(@class,'visible')]//tbody/tr"
    )
    
    row = rows[i]
    cells = row.find_elements(By.TAG_NAME, "td")
    
    def clean_num(text):
        clean = re.sub(r"[^\d\.\-]", "", text)
        return float(clean) if clean else 0.0
    
    time_str = cells[0].text.strip()
    
    if not re.match(r"^\d{2}:\d{2}$", time_str):
        continue
    
    timestamp = datetime.combine(
        date_obj,
        datetime.strptime(time_str, "%H:%M").time()
    )
    
    day_data.append({
        "timestamp": timestamp,
        "temperature_c": clean_num(cells[1].text),
        "humidity_percent": clean_num(cells[2].text),
        "pressure_hpa": clean_num(cells[3].text),
        "wind_kmh": clean_num(cells[4].text),
        "precipitation_mm": clean_num(cells[6].text)
    })

In [11]:
print(day_data)

[{'timestamp': datetime.datetime(2026, 1, 20, 0, 0), 'temperature_c': 13.1, 'humidity_percent': 87.0, 'pressure_hpa': 0.0, 'wind_kmh': 0.1, 'precipitation_mm': 0.0}, {'timestamp': datetime.datetime(2026, 1, 20, 1, 0), 'temperature_c': 12.7, 'humidity_percent': 87.0, 'pressure_hpa': 0.0, 'wind_kmh': 0.1, 'precipitation_mm': 0.0}, {'timestamp': datetime.datetime(2026, 1, 20, 2, 0), 'temperature_c': 12.0, 'humidity_percent': 86.0, 'pressure_hpa': 0.0, 'wind_kmh': 0.1, 'precipitation_mm': 0.0}, {'timestamp': datetime.datetime(2026, 1, 20, 3, 0), 'temperature_c': 11.7, 'humidity_percent': 86.0, 'pressure_hpa': 0.0, 'wind_kmh': 0.2, 'precipitation_mm': 0.0}, {'timestamp': datetime.datetime(2026, 1, 20, 4, 0), 'temperature_c': 11.5, 'humidity_percent': 86.0, 'pressure_hpa': 0.0, 'wind_kmh': 0.2, 'precipitation_mm': 0.0}, {'timestamp': datetime.datetime(2026, 1, 20, 5, 0), 'temperature_c': 11.7, 'humidity_percent': 86.0, 'pressure_hpa': 0.0, 'wind_kmh': 0.2, 'precipitation_mm': 0.0}, {'timesta

In [12]:
table = wait.until(
    EC.visibility_of_element_located(
        (By.XPATH, "//div[contains(@class,'card') and contains(@class,'visible')]//table")
    )
)

headers = table.find_elements(By.XPATH, ".//thead/tr/th")

print("\nTable Headers:")
for i, header in enumerate(headers):
    print(f"  Column {i}: {header.text}")

rows = table.find_elements(By.XPATH, ".//tbody/tr")
first_data_row = None

for row in rows:
    cells = row.find_elements(By.TAG_NAME, "td")
    time_str = cells[0].text.strip()
    
    if re.match(r"^\d{2}:\d{2}$", time_str):
        first_data_row = cells
        break

if first_data_row:
    print("\nFirst Data Row (with units):")
    for i, cell in enumerate(first_data_row):
        text = cell.text.strip()
        print(f"  Column {i}: '{text}'")
        
        value_match = re.search(r'([\d\.\-]+)\s*([^\d\s]+)?', text)
        if value_match:
            value = value_match.group(1)
            unit = value_match.group(2) if value_match.group(2) else "no unit"
            print(f"    → Value: {value}, Unit: {unit}")

print("\n" + "=" * 80)
print("Unit Extraction Patterns:")
print("=" * 80)

if first_data_row:
    patterns = {
        'Temperature': (1, r'([\-\d\.]+)\s*°?C'),
        'Humidity': (2, r'([\d\.]+)\s*%'),
        'Pressure': (3, r'([\d\.]+)\s*hPa'),
        'Wind Speed': (4, r'([\d\.]+)\s*km/h'),
        'Gust Speed': (5, r'([\d\.]+)\s*km/h'),
        'Precipitation': (6, r'([\d\.]+)\s*mm')
    }
    
    for name, (idx, pattern) in patterns.items():
        text = first_data_row[idx].text.strip()
        match = re.search(pattern, text)
        if match:
            value = match.group(1)
            print(f"\n{name}:")
            print(f"  Raw text: '{text}'")
            print(f"  Extracted value: {value}")
            print(f"  Pattern: {pattern}")


Table Headers:
  Column 0: Time
  Column 1: Temperature
  Column 2: Humidity
  Column 3: Air pressure
  Column 4: Wind speed
  Column 5: Gust speed
  Column 6: Precipitation

First Data Row (with units):
  Column 0: '00:00'
    → Value: 00, Unit: :
  Column 1: '13.1 °F'
    → Value: 13.1, Unit: °F
  Column 2: '87.0%'
    → Value: 87.0, Unit: %
  Column 3: '0.00 inHg'
    → Value: 0.00, Unit: inHg
  Column 4: '0.1 mph'
    → Value: 0.1, Unit: mph
  Column 5: '0.1 mph'
    → Value: 0.1, Unit: mph
  Column 6: '0.00 in'
    → Value: 0.00, Unit: in

Unit Extraction Patterns:

Humidity:
  Raw text: '87.0%'
  Extracted value: 87.0
  Pattern: ([\d\.]+)\s*%


In [10]:
def scrape_one_day(driver, wait):
    date_element = wait.until(
        EC.visibility_of_element_located(
            (By.XPATH, "//div[contains(@class,'date') and contains(@class,'visible')]//ion-text")
        )
    )
    date_text = date_element.text
    date_obj = datetime.strptime(date_text, "%B %d, %Y").date()
    
    rows_count = len(
        driver.find_elements(
            By.XPATH,
            "//div[contains(@class,'card') and contains(@class,'visible')]//tbody/tr"
        )
    )
    
    day_data = []
    
    for i in range(rows_count):
        rows = driver.find_elements(
            By.XPATH,
            "//div[contains(@class,'card') and contains(@class,'visible')]//tbody/tr"
        )
        
        row = rows[i]
        cells = row.find_elements(By.TAG_NAME, "td")
        
        def clean_num(text):
            clean = re.sub(r"[^\d\.\-]", "", text)
            return float(clean) if clean else 0.0
        
        time_str = cells[0].text.strip()
        
        if not re.match(r"^\d{2}:\d{2}$", time_str):
            continue
        
        timestamp = datetime.combine(
            date_obj,
            datetime.strptime(time_str, "%H:%M").time()
        )
        
        day_data.append({
            "timestamp": timestamp,
            "temperature_c": clean_num(cells[1].text),
            "humidity_percent": clean_num(cells[2].text),
            "pressure_hpa": clean_num(cells[3].text),
            "wind_kmh": clean_num(cells[4].text),
            "precipitation_mm": clean_num(cells[6].text)
        })

    return date_obj, day_data


In [38]:
def go_to_previous_day(driver, wait, current_date_text):
    prev_button = wait.until(
        EC.element_to_be_clickable(
            (By.XPATH, "//div[contains(@class,'date') and contains(@class,'visible')]//ion-buttons[contains(@class,'left')]//ion-button")
        )
    )

    prev_button.click()

    # Wait until date text changes
    wait.until(
        lambda d: d.find_element(
            By.XPATH,
            "//div[contains(@class,'date') and contains(@class,'visible')]//ion-text"
        ).text != current_date_text
    )


In [39]:
# driver.get(urls['table'])

all_data = []
dates_collected = []
days = 14
for i in range(days):
    date_element = wait.until(
        EC.visibility_of_element_located(
            (By.XPATH, "//div[contains(@class,'date') and contains(@class,'visible')]//ion-text")
        )
    )
    current_date_text = date_element.text

    date_obj, day_data = scrape_one_day(driver, wait)
    dates_collected.append(date_obj)
    all_data.extend(day_data)

    print(f"Collected {len(day_data)} rows for {date_obj}")

    if i < (days-1):
        go_to_previous_day(driver, wait, current_date_text)

# driver.quit()


Collected 18 rows for 2026-01-19
Collected 18 rows for 2026-01-18
Collected 24 rows for 2026-01-17
Collected 24 rows for 2026-01-16
Collected 24 rows for 2026-01-15
Collected 24 rows for 2026-01-14
Collected 24 rows for 2026-01-13
Collected 24 rows for 2026-01-12
Collected 24 rows for 2026-01-11
Collected 24 rows for 2026-01-10
Collected 24 rows for 2026-01-09
Collected 24 rows for 2026-01-08
Collected 24 rows for 2026-01-07
Collected 24 rows for 2026-01-06


### Computing antecedent rainfall

In [40]:
def antecedent_rainfall(data, decay=0.84):
    data_sorted = sorted(data, key=lambda x: x["timestamp"], reverse=True)

    rain_eff = 0.0
    last_day = data_sorted[0]["timestamp"].date()

    for row in data_sorted:
        days_ago = (last_day - row["timestamp"].date()).days
        rain_eff += row["precipitation_mm"] * (decay ** days_ago)

    return rain_eff


In [41]:
ra_eff = antecedent_rainfall(all_data)
print(f"Effective antecedent rainfall: {ra_eff:.3f} mm")

Effective antecedent rainfall: 0.065 mm


In [20]:
import sys
from pathlib import Path
# sys.path.append(str(Path(__file__).parent.parent))

# from config.database import load_config
import psycopg2


def show_sample_data():
    """Show sample data to identify unit issues"""
    
    # config = load_config()
    
    with psycopg2.connect(**config) as conn:
        with conn.cursor() as cur:
            cur.execute("""
                SELECT timestamp, temperature_c, humidity_percent, pressure_hpa, 
                       wind_kmh, precipitation_mm
                FROM weather_data_hourly
                ORDER BY timestamp DESC
                LIMIT 10
            """)
            
            rows = cur.fetchall()
            
            print("Current Database Values (Last 10 records):")
            print(f"{'Timestamp':<20} {'Temp(C)':<10} {'Humid(%)':<10} {'Press(hPa)':<12} {'Wind(km/h)':<12} {'Precip(mm)':<12}")
            
            for row in rows:
                print(f"{str(row[0]):<20} {row[1]:<10.1f} {row[2]:<10.1f} {row[3]:<12.1f} {row[4]:<12.1f} {row[5]:<12.1f}")


def convert_temperature(from_unit='F'):
    """
    Convert temperature to Celsius
    
    Args:
        from_unit: 'F' (Fahrenheit) or 'K' (Kelvin)
    """
    
    # config = load_config()
    
    print(f"\nConverting temperature from {from_unit} to Celsius...")
    
    with psycopg2.connect(**config) as conn:
        with conn.cursor() as cur:
            
            cur.execute("CREATE TABLE IF NOT EXISTS weather_data_backup AS SELECT * FROM weather_data_hourly")
            print("Backup created")
            
            if from_unit == 'F':
                cur.execute("""
                    UPDATE weather_data_hourly 
                    SET temperature_c = (temperature_c - 32) * 5.0/9.0
                """)
            elif from_unit == 'K':
                cur.execute("""
                    UPDATE weather_data_hourly 
                    SET temperature_c = temperature_c - 273.15
                """)
            
            conn.commit()
            print(f"Temperature converted from {from_unit} to Celsius")


def convert_pressure(from_unit='inHg'):
    """
    Convert pressure to hPa
    
    Args:
        from_unit: 'inHg' (inches of mercury) or 'atm' (atmospheres)
    """
    
    # config = load_config()
    
    print(f"\nConverting pressure from {from_unit} to hPa...")
    
    with psycopg2.connect(**config) as conn:
        with conn.cursor() as cur:
            
            if from_unit == 'inHg':
                cur.execute("""
                    UPDATE weather_data_hourly 
                    SET pressure_hpa = pressure_hpa * 33.8639
                """)
            elif from_unit == 'atm':
                cur.execute("""
                    UPDATE weather_data_hourly 
                    SET pressure_hpa = pressure_hpa * 1013.25
                """)
            
            conn.commit()
            print(f"Pressure converted from {from_unit} to hPa")


def convert_wind_speed(from_unit='m/s'):
    """
    Convert wind speed to km/h
    
    Args:
        from_unit: 'm/s' (meters per second), 'mph' (miles per hour), or 'kn' (knots)
    """
    
    # config = load_config()
    
    print(f"\nConverting wind speed from {from_unit} to km/h...")
    
    with psycopg2.connect(**config) as conn:
        with conn.cursor() as cur:
            
            if from_unit == 'm/s':
                cur.execute("""
                    UPDATE weather_data_hourly 
                    SET wind_kmh = wind_kmh * 3.6
                """)
            elif from_unit == 'mph':
                cur.execute("""
                    UPDATE weather_data_hourly 
                    SET wind_kmh = wind_kmh * 1.60934
                """)
            elif from_unit == 'kn':
                cur.execute("""
                    UPDATE weather_data_hourly 
                    SET wind_kmh = wind_kmh * 1.852
                """)
            
            conn.commit()
            print(f"Wind speed converted from {from_unit} to km/h")


def convert_precipitation(from_unit='in'):
    """
    Convert precipitation to mm
    
    Args:
        from_unit: 'in' (inches) or 'cm' (centimeters)
    """
    
    # config = load_config()
    
    print(f"\nConverting precipitation from {from_unit} to mm...")
    
    with psycopg2.connect(**config) as conn:
        with conn.cursor() as cur:
            
            if from_unit == 'in':
                cur.execute("""
                    UPDATE weather_data_hourly 
                    SET precipitation_mm = precipitation_mm * 25.4
                """)
            elif from_unit == 'cm':
                cur.execute("""
                    UPDATE weather_data_hourly 
                    SET precipitation_mm = precipitation_mm * 10
                """)
            
            conn.commit()
            print(f"Precipitation converted from {from_unit} to mm")


def restore_backup():
    """Restore data from backup if conversion went wrong"""
    
    # config = load_config()
    
    print("\nRestoring from backup...")
    
    response = input("Are you sure? This will overwrite current data (y/n): ")
    
    if response.lower() == 'y':
        with psycopg2.connect(**config) as conn:
            with conn.cursor() as cur:
                cur.execute("DROP TABLE IF EXISTS weather_data_hourly")
                cur.execute("ALTER TABLE weather_data_backup RENAME TO weather_data_hourly")
                conn.commit()
                print("Data restored from backup")
    else:
        print("Restore cancelled")

In [21]:
show_sample_data()

Current Database Values (Last 10 records):
Timestamp            Temp(C)    Humid(%)   Press(hPa)   Wind(km/h)   Precip(mm)  
2026-01-20 07:00:00  12.0       86.0       0.0          0.0          0.0         
2026-01-20 06:00:00  11.8       86.0       0.0          0.2          0.0         
2026-01-20 05:00:00  11.7       86.0       0.0          0.2          0.0         
2026-01-20 04:00:00  11.5       86.0       0.0          0.2          0.0         
2026-01-20 03:00:00  11.7       86.0       0.0          0.2          0.0         
2026-01-20 02:00:00  12.0       86.0       0.0          0.1          0.0         
2026-01-20 01:00:00  12.7       87.0       0.0          0.1          0.0         
2026-01-20 00:00:00  13.1       87.0       0.0          0.1          0.0         
2026-01-19 23:00:00  13.8       87.0       0.0          0.2          0.0         
2026-01-19 22:00:00  14.9       87.0       0.0          0.1          0.0         


In [22]:
convert_temperature('F')
convert_pressure('inHg')
convert_wind_speed('mph')
convert_precipitation('in')
show_sample_data()


Converting temperature from F to Celsius...
Backup created
Temperature converted from F to Celsius

Converting pressure from inHg to hPa...
Pressure converted from inHg to hPa

Converting wind speed from mph to km/h...
Wind speed converted from mph to km/h

Converting precipitation from in to mm...
Precipitation converted from in to mm
Current Database Values (Last 10 records):
Timestamp            Temp(C)    Humid(%)   Press(hPa)   Wind(km/h)   Precip(mm)  
2026-01-20 07:00:00  -11.1      86.0       0.0          0.0          0.0         
2026-01-20 06:00:00  -11.2      86.0       0.0          0.3          0.0         
2026-01-20 05:00:00  -11.3      86.0       0.0          0.3          0.0         
2026-01-20 04:00:00  -11.4      86.0       0.0          0.3          0.0         
2026-01-20 03:00:00  -11.3      86.0       0.0          0.3          0.0         
2026-01-20 02:00:00  -11.1      86.0       0.0          0.2          0.0         
2026-01-20 01:00:00  -10.7      87.0       0

In [ ]:

def interactive_conversion():
    """Interactive unit conversion menu"""
    
    print("\n" + "=" * 80)
    print("WEATHER DATA UNIT CONVERTER")
    print("=" * 80)
    
    
    
    print("\n\nWhat needs to be converted?")
    print("1. Temperature")
    print("2. Pressure")
    print("3. Wind Speed")
    print("4. Precipitation")
    print("5. Show data again")
    print("6. Restore from backup")
    print("0. Exit")
    
    choice = input("\nEnter choice: ")
    
    if choice == '1':
        print("\nTemperature currently in:")
        print("F - Fahrenheit")
        print("K - Kelvin")
        from_unit = input("From unit: ").upper()
        
        if from_unit in ['F', 'K']:
            convert_temperature(from_unit)
            show_sample_data()
        else:
            print("Invalid unit")
    
    elif choice == '2':
        print("\nPressure currently in:")
        print("inHg - Inches of mercury")
        print("atm - Atmospheres")
        from_unit = input("From unit: ")
        
        if from_unit in ['inHg', 'atm']:
            convert_pressure(from_unit)
            show_sample_data()
        else:
            print("Invalid unit")
    
    elif choice == '3':
        print("\nWind speed currently in:")
        print("m/s - Meters per second")
        print("mph - Miles per hour")
        print("kn - Knots")
        from_unit = input("From unit: ")
        
        if from_unit in ['m/s', 'mph', 'kn']:
            convert_wind_speed(from_unit)
            show_sample_data()
        else:
            print("Invalid unit")
    
    elif choice == '4':
        print("\nPrecipitation currently in:")
        print("in - Inches")
        print("cm - Centimeters")
        from_unit = input("From unit: ")
        
        if from_unit in ['in', 'cm']:
            convert_precipitation(from_unit)
            show_sample_data()
        else:
            print("Invalid unit")
    
    elif choice == '5':
        show_sample_data()
        interactive_conversion()
    
    elif choice == '6':
        restore_backup()
        show_sample_data()
    
    elif choice == '0':
        print("Exiting...")
        return
    
    else:
        print("Invalid choice")
    
    if choice in ['1', '2', '3', '4']:
        response = input("\nConvert another field? (y/n): ")
        if response.lower() == 'y':
            interactive_conversion()



# interactive_conversion()